In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

from sklearn.model_selection import train_test_split

x_train_all, x_test , y_train_all, y_test = train_test_split(
    housing.data,housing.target,random_state = 7)
x_train , x_valid , y_train , y_valid = train_test_split(
    x_train_all,y_train_all,random_state = 11)

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [7]:
# model = keras.models.Sequential([
#     keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
#     keras.layers.Dense(1),
# ])
# input = keras.layers.Input(shape=x_train.shape[1:])
# hidden1 = keras.layers.Dense(30,activation='relu')(input)
# hidden2 = keras.layers.Dense(30,activation='relu')(hidden1)

# concat = keras.layers.concatenate([input,hidden2])
# output = keras.layers.Dense(1)(concat)

# model = keras.models.Model(inputs = [input],outputs = [output])
class WideDeep(keras.models.Model):
    def __init__(self):
        super(WideDeep,self).__init__()
        self.hidden1_layer = keras.layers.Dense(30,activation='relu')
        self.hidden2_layer = keras.layers.Dense(30,activation='relu')
        self.output_layer = keras.layers.Dense(1)
        
    def call(self,input):
        hidden1 = self.hidden1_layer(input)
        hidden2 = self.hidden2_layer(hidden1)
        concat = keras.layers.concatenate([input,hidden2])
        output = self.output_layer(concat)
        return output
model = WideDeep()
model.build(input_shape = (None,8))

model.summary()
model.compile(loss='mean_squared_error',optimizer = 'adamax')#用sgd无法收敛
callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=1e-2)]

Model: "wide_deep_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              multiple                  270       
_________________________________________________________________
dense_10 (Dense)             multiple                  930       
_________________________________________________________________
dense_11 (Dense)             multiple                  39        
Total params: 1,239
Trainable params: 1,239
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(x_train_scaled,y_train,validation_data=(x_valid_scaled,y_valid),
                   epochs=100,callbacks=callbacks)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 109us/sample - loss: 1.6258 - val_loss: 0.9705
Epoch 2/100
11610/11610 [==============================] - 1s 53us/sample - loss: 0.7202 - val_loss: 0.7661
Epoch 3/100
11610/11610 [==============================] - 1s 51us/sample - loss: 0.6076 - val_loss: 0.6546
Epoch 4/100
11610/11610 [==============================] - 1s 50us/sample - loss: 0.5341 - val_loss: 0.5764
Epoch 5/100
11610/11610 [==============================] - 1s 50us/sample - loss: 0.4868 - val_loss: 0.5188
Epoch 6/100
11610/11610 [==============================] - 1s 51us/sample - loss: 0.4528 - val_loss: 0.4834
Epoch 7/100
11610/11610 [==============================] - 1s 51us/sample - loss: 0.4295 - val_loss: 0.4568
Epoch 8/100
11610/11610 [==============================] - 1s 51us/sample - loss: 0.4129 - val_loss: 0.4399
Epoch 9/100
11610/11610 [==============================] - 1s 51us/sample - loss: 0.40